In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image

# ============== CONFIGURACIÓN ==============
DATASET_PATH = r"C:\Users\Usuario\.cache\kagglehub\datasets\philosopher0808\real-vs-ai-generated-faces-dataset\versions\1\dataset\dataset"
IMG_SIZE = 224  # Tamaño óptimo para transfer learning
BATCH_SIZE = 32
NUM_CLASSES = 2
EPOCHS = 50

# ============== DATA GENERATORS ==============
print("Configurando generadores de datos...")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]  # Variación de brillo
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=f"{DATASET_PATH}/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    directory=f"{DATASET_PATH}/validate",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    directory=f"{DATASET_PATH}/test",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Guardar nombres de clases
class_names = list(train_generator.class_indices.keys())
print(f"Clases detectadas: {class_names}")
print(f"Total imágenes entrenamiento: {train_generator.samples}")
print(f"Total imágenes validación: {val_generator.samples}")
print(f"Total imágenes test: {test_generator.samples}")

# ============== CREAR MODELO CON TRANSFER LEARNING ==============
print("\nCreando modelo con MobileNetV2...")

# Cargar modelo base preentrenado
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Congelar las capas del modelo base inicialmente
base_model.trainable = False

# Crear el modelo completo
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax')
], name='AI_Image_Detector')

model.summary()

# ============== COMPILAR MODELO ==============
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# ============== CALLBACKS ==============
callbacks = [
    # Detener si no mejora en 7 épocas
    EarlyStopping(
        monitor='val_loss',
        patience=7,
        restore_best_weights=True,
        verbose=1
    ),
    # Guardar el mejor modelo
    ModelCheckpoint(
        'best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    # Reducir learning rate si se estanca
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

# ============== ENTRENAMIENTO ==============
print("\n========== FASE 1: Entrenamiento con capas congeladas ==========")
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

# ============== FINE-TUNING ==============
print("\n========== FASE 2: Fine-tuning (descongelando capas) ==========")

# Descongelar las últimas capas del modelo base
base_model.trainable = True

# Congelar solo las primeras capas
for layer in base_model.layers[:100]:
    layer.trainable = False

# Recompilar con learning rate más bajo
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Continuar entrenamiento
history_fine = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    initial_epoch=len(history.history['loss']),
    verbose=1
)

# Combinar historiales
for key in history.history.keys():
    history.history[key].extend(history_fine.history[key])

# ============== EVALUACIÓN ==============
print("\n========== EVALUACIÓN EN TEST SET ==========")
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test AUC: {test_auc:.4f}")

# Guardar modelo final
model.save('final_model.keras')
print("\nModelo guardado como 'final_model.keras'")

# ============== VISUALIZACIÓN DE RESULTADOS ==============
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], 'b', label='Train')
axes[0].plot(history.history['val_accuracy'], 'r', label='Validation')
axes[0].set_xlabel('Épocas')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy durante entrenamiento')
axes[0].legend()
axes[0].grid(True)

# Loss
axes[1].plot(history.history['loss'], 'b', label='Train')
axes[1].plot(history.history['val_loss'], 'r', label='Validation')
axes[1].set_xlabel('Épocas')
axes[1].set_ylabel('Loss')
axes[1].set_title('Loss durante entrenamiento')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# ============== FUNCIÓN DE PREDICCIÓN ==============
def predict_image(img_path, model, class_names):
    """
    Predice si una imagen es real o generada por IA
    
    Args:
        img_path: Ruta de la imagen
        model: Modelo entrenado
        class_names: Lista con nombres de las clases
        
    Returns:
        predicted_class: Clase predicha
        confidence: Confianza de la predicción (%)
    """
    # Cargar y preprocesar imagen
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Predecir
    prediction = model.predict(img_array, verbose=0)
    predicted_idx = np.argmax(prediction)
    predicted_class = class_names[predicted_idx]
    confidence = prediction[0][predicted_idx] * 100
    
    # Visualizar
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.title(f"Predicción: {predicted_class}\nConfianza: {confidence:.2f}%", 
              fontsize=14, fontweight='bold')
    plt.axis('off')
    
    # Añadir barra de probabilidades
    plt.text(10, img.size[1] - 10, 
             f"{class_names[0]}: {prediction[0][0]*100:.1f}%\n{class_names[1]}: {prediction[0][1]*100:.1f}%",
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
             fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n{'='*50}")
    print(f"Imagen: {img_path}")
    print(f"Predicción: {predicted_class}")
    print(f"Confianza: {confidence:.2f}%")
    print(f"Probabilidades detalladas:")
    for i, class_name in enumerate(class_names):
        print(f"  - {class_name}: {prediction[0][i]*100:.2f}%")
    print(f"{'='*50}\n")
    
    return predicted_class, confidence

# ============== PREDICCIONES DE EJEMPLO ==============
print("\n========== PREDICCIONES EN IMÁGENES DE PRUEBA ==========")

# Lista de imágenes a probar
test_images = [
    r"C:\Users\Usuario\Downloads\IA1.jpg",
    r"C:\Users\Usuario\Downloads\IA2.jpg",
    r"C:\Users\Usuario\Downloads\IA3.jpg",
    r"C:\Users\Usuario\Downloads\IA4.jpg"
]

# Predecir cada imagen
results = []
for img_path in test_images:
    try:
        predicted_class, confidence = predict_image(img_path, model, class_names)
        results.append({
            'path': img_path,
            'prediction': predicted_class,
            'confidence': confidence
        })
    except Exception as e:
        print(f"Error al procesar {img_path}: {e}")

# Resumen de resultados
print("\n========== RESUMEN DE PREDICCIONES ==========")
for i, result in enumerate(results, 1):
    print(f"{i}. {result['path'].split('\\')[-1]}: {result['prediction']} ({result['confidence']:.2f}%)")

print("\n✓ Proceso completado exitosamente")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image

# ============== CONFIGURACIÓN ==============
import os

DATASET_PATH = r"C:\Users\Usuario\.cache\kagglehub\datasets\philosopher0808\real-vs-ai-generated-faces-dataset\versions\1\dataset\dataset"
IMG_SIZE = 224  # Tamaño óptimo para transfer learning
BATCH_SIZE = 32
NUM_CLASSES = 2
EPOCHS = 50

# ============== VERIFICAR Y EXPLORAR DATASET ==============
print("Verificando estructura del dataset...")

# Verificar si existe el directorio base
if not os.path.exists(DATASET_PATH):
    print(f"❌ ERROR: No existe el directorio: {DATASET_PATH}")
    print("\nBuscando la ruta correcta...")
    
    # Intentar encontrar el dataset
    base_path = r"C:\Users\Usuario\.cache\kagglehub\datasets\philosopher0808"
    if os.path.exists(base_path):
        print(f"✓ Directorio base encontrado: {base_path}")
        print("\nContenido del directorio:")
        for root, dirs, files in os.walk(base_path):
            level = root.replace(base_path, '').count(os.sep)
            indent = ' ' * 2 * level
            print(f"{indent}{os.path.basename(root)}/")
            subindent = ' ' * 2 * (level + 1)
            for file in files[:5]:  # Solo primeros 5 archivos
                print(f"{subindent}{file}")
            if len(files) > 5:
                print(f"{subindent}... y {len(files)-5} archivos más")
    else:
        print(f"❌ No se encuentra ni siquiera: {base_path}")
        print("\n💡 Solución: Verifica que hayas descargado el dataset correctamente desde Kaggle")
    
    raise FileNotFoundError(f"No se encuentra el dataset en {DATASET_PATH}")

print(f"✓ Directorio base existe: {DATASET_PATH}")

# Explorar estructura
print("\nEstructura del dataset:")
for item in os.listdir(DATASET_PATH):
    item_path = os.path.join(DATASET_PATH, item)
    if os.path.isdir(item_path):
        print(f"  📁 {item}/")
        # Ver contenido de subdirectorios
        try:
            subdirs = os.listdir(item_path)
            for subdir in subdirs[:5]:  # Primeros 5
                subdir_path = os.path.join(item_path, subdir)
                if os.path.isdir(subdir_path):
                    n_files = len([f for f in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, f))])
                    print(f"      📂 {subdir}/ ({n_files} archivos)")
        except:
            pass

# Detectar si los subdirectorios son train/validate/test o training/validation/testing
possible_train_names = ['train', 'training', 'Train', 'TRAIN']
possible_val_names = ['validate', 'validation', 'val', 'Validate', 'VALIDATE']
possible_test_names = ['test', 'testing', 'Test', 'TEST']

train_dir = None
val_dir = None
test_dir = None

for item in os.listdir(DATASET_PATH):
    item_lower = item.lower()
    if item_lower in [n.lower() for n in possible_train_names]:
        train_dir = os.path.join(DATASET_PATH, item)
    elif item_lower in [n.lower() for n in possible_val_names]:
        val_dir = os.path.join(DATASET_PATH, item)
    elif item_lower in [n.lower() for n in possible_test_names]:
        test_dir = os.path.join(DATASET_PATH, item)

if not all([train_dir, val_dir, test_dir]):
    print("\n⚠️  ADVERTENCIA: No se encontraron los directorios esperados (train/validate/test)")
    print("Directorios encontrados:")
    print(f"  Train: {train_dir if train_dir else '❌ NO ENCONTRADO'}")
    print(f"  Validation: {val_dir if val_dir else '❌ NO ENCONTRADO'}")
    print(f"  Test: {test_dir if test_dir else '❌ NO ENCONTRADO'}")
    print("\n💡 El dataset puede tener una estructura diferente.")
    print("Por favor, indica cuál es la estructura correcta del dataset.")
    raise FileNotFoundError("Estructura de directorios no compatible")

print(f"\n✓ Directorios encontrados:")
print(f"  Train: {train_dir}")
print(f"  Validation: {val_dir}")
print(f"  Test: {test_dir}")

# ============== DATA GENERATORS ==============
print("\nConfigurando generadores de datos...")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]  # Variación de brillo
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=train_dir,  # Usar la ruta detectada automáticamente
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    directory=val_dir,  # Usar la ruta detectada automáticamente
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    directory=test_dir,  # Usar la ruta detectada automáticamente
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Guardar nombres de clases
class_names = list(train_generator.class_indices.keys())
print(f"Clases detectadas: {class_names}")
print(f"Total imágenes entrenamiento: {train_generator.samples}")
print(f"Total imágenes validación: {val_generator.samples}")
print(f"Total imágenes test: {test_generator.samples}")

# ============== CREAR MODELO CON TRANSFER LEARNING ==============
print("\nCreando modelo con MobileNetV2...")

# Cargar modelo base preentrenado
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Congelar las capas del modelo base inicialmente
base_model.trainable = False

# Crear el modelo completo
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax')
], name='AI_Image_Detector')

model.summary()

# ============== COMPILAR MODELO ==============
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# ============== CALLBACKS ==============
callbacks = [
    # Detener si no mejora en 7 épocas
    EarlyStopping(
        monitor='val_loss',
        patience=7,
        restore_best_weights=True,
        verbose=1
    ),
    # Guardar el mejor modelo
    ModelCheckpoint(
        'best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    # Reducir learning rate si se estanca
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

# ============== ENTRENAMIENTO ==============
print("\n========== FASE 1: Entrenamiento con capas congeladas ==========")
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

# ============== FINE-TUNING ==============
print("\n========== FASE 2: Fine-tuning (descongelando capas) ==========")

# Descongelar las últimas capas del modelo base
base_model.trainable = True

# Congelar solo las primeras capas
for layer in base_model.layers[:100]:
    layer.trainable = False

# Recompilar con learning rate más bajo
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Continuar entrenamiento
history_fine = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    initial_epoch=len(history.history['loss']),
    verbose=1
)

# Combinar historiales
for key in history.history.keys():
    history.history[key].extend(history_fine.history[key])

# ============== EVALUACIÓN ==============
print("\n========== EVALUACIÓN EN TEST SET ==========")
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test AUC: {test_auc:.4f}")

# Guardar modelo final
model.save('final_model.keras')
print("\nModelo guardado como 'final_model.keras'")

# ============== VISUALIZACIÓN DE RESULTADOS ==============
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], 'b', label='Train')
axes[0].plot(history.history['val_accuracy'], 'r', label='Validation')
axes[0].set_xlabel('Épocas')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy durante entrenamiento')
axes[0].legend()
axes[0].grid(True)

# Loss
axes[1].plot(history.history['loss'], 'b', label='Train')
axes[1].plot(history.history['val_loss'], 'r', label='Validation')
axes[1].set_xlabel('Épocas')
axes[1].set_ylabel('Loss')
axes[1].set_title('Loss durante entrenamiento')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# ============== FUNCIÓN DE PREDICCIÓN ==============
def predict_image(img_path, model, class_names):
    """
    Predice si una imagen es real o generada por IA
    
    Args:
        img_path: Ruta de la imagen
        model: Modelo entrenado
        class_names: Lista con nombres de las clases
        
    Returns:
        predicted_class: Clase predicha
        confidence: Confianza de la predicción (%)
    """
    # Cargar y preprocesar imagen
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Predecir
    prediction = model.predict(img_array, verbose=0)
    predicted_idx = np.argmax(prediction)
    predicted_class = class_names[predicted_idx]
    confidence = prediction[0][predicted_idx] * 100
    
    # Visualizar
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.title(f"Predicción: {predicted_class}\nConfianza: {confidence:.2f}%", 
              fontsize=14, fontweight='bold')
    plt.axis('off')
    
    # Añadir barra de probabilidades
    plt.text(10, img.size[1] - 10, 
             f"{class_names[0]}: {prediction[0][0]*100:.1f}%\n{class_names[1]}: {prediction[0][1]*100:.1f}%",
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
             fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n{'='*50}")
    print(f"Imagen: {img_path}")
    print(f"Predicción: {predicted_class}")
    print(f"Confianza: {confidence:.2f}%")
    print(f"Probabilidades detalladas:")
    for i, class_name in enumerate(class_names):
        print(f"  - {class_name}: {prediction[0][i]*100:.2f}%")
    print(f"{'='*50}\n")
    
    return predicted_class, confidence

# ============== PREDICCIONES DE EJEMPLO ==============
print("\n========== PREDICCIONES EN IMÁGENES DE PRUEBA ==========")

# Lista de imágenes a probar
test_images = [
    r"C:\Users\Usuario\Downloads\IA1.jpg",
    r"C:\Users\Usuario\Downloads\IA2.jpg",
    r"C:\Users\Usuario\Downloads\IA3.jpg",
    r"C:\Users\Usuario\Downloads\IA4.jpg"
]

# Predecir cada imagen
results = []
for img_path in test_images:
    try:
        predicted_class, confidence = predict_image(img_path, model, class_names)
        results.append({
            'path': img_path,
            'prediction': predicted_class,
            'confidence': confidence
        })
    except Exception as e:
        print(f"Error al procesar {img_path}: {e}")

# Resumen de resultados
print("\n========== RESUMEN DE PREDICCIONES ==========")
for i, result in enumerate(results, 1):
    print(f"{i}. {result['path'].split('\\')[-1]}: {result['prediction']} ({result['confidence']:.2f}%)")

print("\n✓ Proceso completado exitosamente")